# premium ohlc

## unit functions

In [50]:
import json, os, gzip
import datetime
import requests
import pandas as pd
import matplotlib.pyplot as plt

def date_to_str(date):
    return date.strftime('%Y-%m-%d')

def date_to_ymd_str(date):
    # return list of yyy, mm, dd as string (2 digit)
    return str(date).split(' ')[0].split('-')

def strTZ_to_date(txt):
    # ex: txt = '2019-08-01T00:00:00.000Z'
    return datetime.datetime.strptime(txt,'%Y-%m-%dT%H:%M:%S.%fZ')

def filename_to_date(filename, symbol):
    datestr = filename.replace(symbol,'').replace('_premium_index','').replace('_index_price','').replace('.csv.gz','')
    return datetime.datetime.strptime(datestr,'%Y-%m-%d')

def get_list_symbols(dataType):
    if dataType in ['premium_index', 'spot_index', 'trading']:
        res = requests.get(f'https://public.bybit.com/{dataType}/')
        list_doc = [doc.split('"')[1][:-1] for doc in res.text.split('href=')[1:]]
        return list_doc

def get_list_filename_premiumIndex(symbol):
    if symbol in get_list_symbols('premium_index'):
        res = requests.get(f'https://public.bybit.com/premium_index/{symbol}/')
        list_doc = [doc.split('"')[1] for doc in res.text.split('href=')[1:]]
        return list_doc

def dowload_monthly_premium_index(symbol='BTCUSD', year=2020, month=5, print_mode=False):      
    date = datetime.datetime(year, month, 1)    
    list_path = []
    if print_mode:
        print(f'{datetime.datetime.now()} downloading data for premium index {symbol} year {year} and month {month}')
    # loop on day
    while date.month == month:    
        url = 'https://public.bybit.com/premium_index/:symbol/:symbol:year-:month-:day_premium_index.csv.gz'
        # create url & path
        params = {
            'symbol': symbol,
            'year': date_to_ymd_str(date)[0],
            'month': date_to_ymd_str(date)[1],
            'day': date_to_ymd_str(date)[2]
        }    
        for key, item in params.items():
            url = url.replace(':'+key, item)
       # request + up^load data
        with open(f'tmp_data/{date_to_ymd_str(date)[2]}.csv.gz', "wb") as f:
            list_path.append(f'tmp_data/{date_to_ymd_str(date)[2]}.csv.gz')
            r = requests.get(url)
            f.write(r.content)            
        date += datetime.timedelta(days=1)
    # extract df month
    df = pd.DataFrame(columns=['start_at', 'symbol', 'period', 'open', 'high', 'low', 'close'])
    for path in list_path:
        df = pd.concat([df, pd.read_csv(path)],axis=0)
        os.remove(path)        
    df = df.reset_index(drop=True).rename(columns={"start_at":"time"}).drop(['symbol', 'period'], axis=1)
    # upload
    path = 'D://Trading/Data/Crypto/Bybit/premium_ohlc/:symbol/1min/:year/:year-:month.csv.gz'
    for key, item in params.items():
        path = path.replace(':'+key, item)
        
    folders_to_create = path[4:].split('/')[:-1]
    tmp_path = 'D://'
    for folder in folders_to_create:
        if folder not in os.listdir(tmp_path):
            os.mkdir(tmp_path+folder)
        tmp_path += folder + '/'
    
    df.to_csv(path,index=False)
        
def init_premium_ohlc():
    
    if 'infos.json' not in os.listdir('D://Trading/Data/Crypto/Bybit'):
        dic = {}
    else:    
        with open('D://Trading/Data/Crypto/Bybit/infos.json', 'r') as f:
            dic = json.load(f)
            f.close()
    
    if 'premium_ohlc' not in dic:
        dic['premium_ohlc'] = {
            "timeframe_str": {
                "1min": {
                    "file_length": "1m",
                    "columns": [
                      "time",
                      "open",
                      "high",
                      "low",
                      "close"
                    ],
                    "timeframe_sec": 60,
                    "hasVolume": false,
                    "unit_time": "s",
                    "is_gz": true,
                    "is_native": true,
                    "description": "ohlc of futures premium, i.e the difference between the perpetual contracts relative to the mark price. premium = 0.000003 => futures price is 0.03% higher than mark price",
                    "dic_year_month": {}
                }
            }
        }
            
    with open('D://Trading/Data/Crypto/Bybit/infos.json', 'w') as f:
        json.dump(dic,f)
        f.close()
        
init_premium_ohlc()

In [93]:
dowload_monthly_premium_index(symbol='BTCUSD', year=2020, month=5, print_mode=True)

2021-03-22 17:48:33.797173 downloading data for premium index BTCUSD year 2020 and month 5


In [37]:
df = pd.read_csv('D://Trading/Data/Crypto/Bybit/premium_ohlc/BTCUSD/1min/2020/2020-05.csv.gz')
df.loc[:,'date'] = pd.to_datetime(df.time, unit='s')
df

,time,open,high,low,close,date
0,1588291200,0.000025,0.000100,0.000025,0.000100,2020-05-01 00:00:00
1,1588291260,0.000100,0.000100,-0.000169,-0.000169,2020-05-01 00:01:00
2,1588291320,-0.000169,-0.000074,-0.000169,-0.000074,2020-05-01 00:02:00
3,1588291380,-0.000074,0.000198,-0.000074,0.000198,2020-05-01 00:03:00
4,1588291440,0.000198,0.000198,-0.000289,-0.000289,2020-05-01 00:04:00
...,...,...,...,...,...,...
44635,1590969300,-0.000162,-0.000039,-0.000162,-0.000039,2020-05-31 23:55:00
44636,1590969360,-0.000039,0.000100,-0.000039,0.000100,2020-05-31 23:56:00
44637,1590969420,0.000100,0.000100,-0.000094,-0.000094,2020-05-31 23:57:00
44638,1590969480,-0.000094,0.000083,-0.000094,0.000083,2020-05-31 23:58:00


## loop

In [52]:
dataType = 'premium_index'
for symbol in get_list_symbols(dataType):
    dic_year_month = {}
    list_filename = get_list_filename_premiumIndex(symbol)
    print(f'{datetime.datetime.now()} - dowloading {symbol} data...')
    for filename in list_filename:
        date = filename_to_date(filename, symbol)
        if date.year not in dic_year_month:
            dic_year_month[date.year] = []
        if date.month not in dic_year_month[date.year]:
            dic_year_month[date.year].append(date.month)
    del dic_year_month[date.year][-1]
            
    for year in dic_year_month:
        for month in dic_year_month[year]:
            dowload_monthly_premium_index(symbol=symbol, year=year, month=month, print_mode=True)            


2021-03-22 19:19:15.636069 - dowloading BTCUSD data...
2021-03-22 19:19:15.641055 downloading data for premium index BTCUSD year 2019 and month 10
2021-03-22 19:19:18.872849 downloading data for premium index BTCUSD year 2019 and month 11
2021-03-22 19:19:31.898088 downloading data for premium index BTCUSD year 2019 and month 12
2021-03-22 19:19:44.357348 downloading data for premium index BTCUSD year 2020 and month 1
2021-03-22 19:19:58.676704 downloading data for premium index BTCUSD year 2020 and month 2
2021-03-22 19:20:10.216188 downloading data for premium index BTCUSD year 2020 and month 3
2021-03-22 19:20:23.045300 downloading data for premium index BTCUSD year 2020 and month 4
2021-03-22 19:20:35.039441 downloading data for premium index BTCUSD year 2020 and month 5
2021-03-22 19:20:47.500486 downloading data for premium index BTCUSD year 2020 and month 6
2021-03-22 19:21:02.608808 downloading data for premium index BTCUSD year 2020 and month 7
2021-03-22 19:21:17.273258 downl

# spot ohlc

## unit functions

In [26]:
import json, os, gzip
import datetime
import requests
import pandas as pd
import matplotlib.pyplot as plt

def date_to_str(date):
    return date.strftime('%Y-%m-%d')

def date_to_ymd_str(date):
    # return list of yyy, mm, dd as string (2 digit)
    return str(date).split(' ')[0].split('-')

def strTZ_to_date(txt):
    # ex: txt = '2019-08-01T00:00:00.000Z'
    return datetime.datetime.strptime(txt,'%Y-%m-%dT%H:%M:%S.%fZ')

def filename_to_date(filename, symbol):
    datestr = filename.replace(symbol,'').replace('_premium_index','').replace('_index_price','').replace('.csv.gz','')
    return datetime.datetime.strptime(datestr,'%Y-%m-%d')

def get_list_symbols(dataType):
    if dataType in ['premium_index', 'spot_index', 'trading']:
        res = requests.get(f'https://public.bybit.com/{dataType}/')
        list_doc = [doc.split('"')[1][:-1] for doc in res.text.split('href=')[1:]]
        return list_doc

def get_list_filename_spotIndex(symbol):
    if symbol in get_list_symbols('spot_index'):
        res = requests.get(f'https://public.bybit.com/spot_index/{symbol}/')
        list_doc = [doc.split('"')[1] for doc in res.text.split('href=')[1:]]
        return list_doc

def dowload_monthly_spot_index(symbol='BTCUSD', year=2020, month=5, print_mode=False):      
    date = datetime.datetime(year, month, 1)    
    list_path = []
    if print_mode:
        print(f'{datetime.datetime.now()} downloading data for spot index {symbol} year {year} and month {month}')
    # loop on day
    while date.month == month:    
        url = 'https://public.bybit.com/spot_index/:symbol/:symbol:year-:month-:day_index_price.csv.gz'
        # create url & path
        params = {
            'symbol': symbol,
            'year': date_to_ymd_str(date)[0],
            'month': date_to_ymd_str(date)[1],
            'day': date_to_ymd_str(date)[2]
        }    
        for key, item in params.items():
            url = url.replace(':'+key, item)
       # request + up^load data
        with open(f'tmp_data/{date_to_ymd_str(date)[2]}.csv.gz', "wb") as f:
            list_path.append(f'tmp_data/{date_to_ymd_str(date)[2]}.csv.gz')
            r = requests.get(url)
            f.write(r.content)            
        date += datetime.timedelta(days=1)
    # extract df month
    df = pd.DataFrame(columns=['start_at', 'symbol', 'period', 'open', 'high', 'low', 'close'])
    for path in list_path:
        df = pd.concat([df, pd.read_csv(path)],axis=0)
        os.remove(path)    
    df = df.reset_index(drop=True).rename(columns={"start_at":"time"}).drop(['symbol', 'period'], axis=1)
    # upload
    path = 'D://Trading/Data/Crypto/Bybit/spot_ohlc/:symbol/1min/:year/:year-:month.csv.gz'
    for key, item in params.items():
        path = path.replace(':'+key, item)
    
    df.to_csv(path,index=False)
        
def init_spot_ohlc():
    
    if 'infos.json' not in os.listdir('D://Trading/Data/Crypto/Bybit'):
        dic = {}
    else:    
        with open('D://Trading/Data/Crypto/Bybit/infos.json', 'r') as f:
            dic = json.load(f)
            f.close()
    
    if 'spot_ohlc' not in dic:
        dic['spot_ohlc'] = {            
            "timeframe_str": {
                "1min": {
                    "file_length": "1m",
                    "columns": [
                      "time",
                      "open",
                      "high",
                      "low",
                      "close"
                    ],
                    "timeframe_sec": 60,
                    "hasVolume": false,
                    "unit_time": "s",
                    "is_gz": true,
                    "is_native": true,
                    "description": "ohlc of futures spot, i.e candlesticks of asset price",
                    "dic_year_month": {}
                }
            }
        }
            
    with open('D://Trading/Data/Crypto/Bybit/infos.json', 'w') as f:
        json.dump(dic,f)
        f.close()
        
init_spot_ohlc()

In [22]:
dowload_monthly_spot_index(symbol='BTCUSD', year=2020, month=5, print_mode=True)

2021-03-22 18:33:56.709749 downloading data for spot index BTCUSD year 2020 and month 5


In [29]:
df = pd.read_csv('D://Trading/Data/Crypto/Bybit/spot_ohlc/BTCUSD/1min/2020/2020-05.csv.gz')
df.loc[:,'date'] = pd.to_datetime(df.time, unit='s')
df

,time,open,high,low,close,date
0,1588291200,8627.36,8648.55,8623.64,8648.19,2020-05-01 00:00:00
1,1588291260,8648.19,8673.70,8648.19,8662.79,2020-05-01 00:01:00
2,1588291320,8662.79,8663.38,8635.14,8635.93,2020-05-01 00:02:00
3,1588291380,8635.93,8650.39,8634.29,8646.25,2020-05-01 00:03:00
4,1588291440,8646.25,8651.45,8638.69,8638.69,2020-05-01 00:04:00
...,...,...,...,...,...,...
44635,1590969300,9466.29,9470.60,9466.29,9468.67,2020-05-31 23:55:00
44636,1590969360,9468.67,9473.03,9468.67,9471.83,2020-05-31 23:56:00
44637,1590969420,9471.83,9471.83,9463.85,9464.69,2020-05-31 23:57:00
44638,1590969480,9464.69,9465.43,9455.67,9456.63,2020-05-31 23:58:00


## loop

In [28]:
dataType = 'spot_index'
for symbol in get_list_symbols(dataType):
    dic_year_month = {}
    list_filename = get_list_filename_spotIndex(symbol)
    print(f'{datetime.datetime.now()} - dowloading {symbol} data...')
    for filename in list_filename:
        date = filename_to_date(filename, symbol)
        if date.year not in dic_year_month:
            dic_year_month[date.year] = []
        if date.month not in dic_year_month[date.year]:
            dic_year_month[date.year].append(date.month)
    del dic_year_month[date.year][-1]
            
    for year in dic_year_month:
        for month in dic_year_month[year]:
            if symbol not in os.listdir('D://Trading/Data/Crypto/Bybit/spot_ohlc'):
                if str(year) not in os.listdir(f'D://Trading/Data/Crypto/Bybit/spot_ohlc/{symbol}/1min') or True:
                    dowload_monthly_spot_index(symbol=symbol, year=year, month=month, print_mode=True)   

2021-03-22 18:43:15.168060 - dowloading BTCUSD data...
2021-03-22 18:43:15.172073 downloading data for spot index BTCUSD year 2019 and month 10
2021-03-22 18:43:32.975429 downloading data for spot index BTCUSD year 2019 and month 11
2021-03-22 18:43:49.298813 downloading data for spot index BTCUSD year 2019 and month 12
2021-03-22 18:44:07.836969 downloading data for spot index BTCUSD year 2020 and month 1
2021-03-22 18:44:25.119595 downloading data for spot index BTCUSD year 2020 and month 2
2021-03-22 18:44:42.702640 downloading data for spot index BTCUSD year 2020 and month 3
2021-03-22 18:45:00.220872 downloading data for spot index BTCUSD year 2020 and month 4
2021-03-22 18:45:17.995735 downloading data for spot index BTCUSD year 2020 and month 5
2021-03-22 18:45:20.977689 downloading data for spot index BTCUSD year 2020 and month 6
2021-03-22 18:45:36.914657 downloading data for spot index BTCUSD year 2020 and month 7
2021-03-22 18:45:55.666085 downloading data for spot index BTC

# Trade history tick

## unit functions

In [59]:
import json, os, gzip
import datetime
import requests
import pandas as pd
import matplotlib.pyplot as plt

def date_to_str(date):
    return date.strftime('%Y-%m-%d')

def date_to_ymd_str(date):
    # return list of yyy, mm, dd as string (2 digit)
    return str(date).split(' ')[0].split('-')

def strTZ_to_date(txt):
    # ex: txt = '2019-08-01T00:00:00.000Z'
    return datetime.datetime.strptime(txt,'%Y-%m-%dT%H:%M:%S.%fZ')

def filename_to_date(filename, symbol):
    datestr = filename.replace(symbol,'').replace('_premium_index','').replace('_index_price','').replace('.csv.gz','')
    return datetime.datetime.strptime(datestr,'%Y-%m-%d')

def get_list_symbols(dataType):
    if dataType in ['premium_index', 'spot_index', 'trading']:
        res = requests.get(f'https://public.bybit.com/{dataType}/')
        list_doc = [doc.split('"')[1][:-1] for doc in res.text.split('href=')[1:]]
        return list_doc
    
def get_list_filename_trading(symbol):
    if symbol in get_list_symbols('trading'):
        res = requests.get(f'https://public.bybit.com/trading/{symbol}/')
        list_doc = [doc.split('"')[1] for doc in res.text.split('href=')[1:]]
        return list_doc

def dowload_monthly_trading(symbol='BTCUSD', year=2020, month=5, print_mode=False):      
    date = datetime.datetime(year, month, 1)    
    list_path = []
    if print_mode:
        print(f'{datetime.datetime.now()} downloading data for trades index {symbol} year {year} and month {month}')
    # test
    tmp_url = ':symbol:year-:month-:day.csv.gz'
    params = {
        'symbol': symbol,
        'year': date_to_ymd_str(date)[0],
        'month': date_to_ymd_str(date)[1],
        'day': date_to_ymd_str(date)[2]
    }    
    for key, item in params.items():
        tmp_url = tmp_url.replace(':'+key, item)
    if tmp_url not in get_list_filename_trading(symbol):
        print(tmp_url, 'not in historical data')
        return 0
    
    # loop on day    
    while date.month == month:    
        url = 'https://public.bybit.com/trading/:symbol/:symbol:year-:month-:day.csv.gz'
        # create url & path
        params = {
            'symbol': symbol,
            'year': date_to_ymd_str(date)[0],
            'month': date_to_ymd_str(date)[1],
            'day': date_to_ymd_str(date)[2]
        }    
        for key, item in params.items():
            url = url.replace(':'+key, item)
       # request + up^load data
        with open(f'tmp_data/{date_to_ymd_str(date)[2]}.csv.gz', "wb") as f:
            list_path.append(f'tmp_data/{date_to_ymd_str(date)[2]}.csv.gz')
            r = requests.get(url)
            f.write(r.content)         
        date += datetime.timedelta(days=1)
    # extract df month
    df = pd.DataFrame(columns=['timestamp', 'symbol', 'side', 'size', 'price', 'tickDirection', 'trdMatchID', 'grossValue', 'homeNotional', 'foreignNotional'])
    for path in list_path:
        df = pd.concat([df, pd.read_csv(path)],axis=0)
        os.remove(path)        
    df = df.reset_index(drop=True).rename(columns={"timestamp":"time"}).drop(['symbol', 'trdMatchID', 'grossValue', 'homeNotional'], axis=1)
    # upload
    path = 'D://Trading/Data/Crypto/Bybit/trades/:symbol/1min/:year/:year-:month.csv.gz'
    for key, item in params.items():
        path = path.replace(':'+key, item)
    
    df.to_csv(path,index=False)
    
    return 1
        
def init_trades():
    
    if 'infos.json' not in os.listdir('D://Trading/Data/Crypto/Bybit'):
        dic = {}
    else:    
        with open('D://Trading/Data/Crypto/Bybit/infos.json', 'r') as f:
            dic = json.load(f)
            f.close()
    
    if 'trades' not in dic:
        dic['trades'] = {
            "file_length": "1m",
            "columns" : ['time', 'open', 'high', 'low', 'close'],
            "timeframe_str": "tick",
            "timeframe_sec": 0,
            "hasVolume" : True,
            "unit_time": 's',
            "is_gz": True,
            "is_native": True,
            "description": "trades granular data, contain all trades made (tick by tick)",
            "dic_year_month": {}
        }
            
    with open('D://Trading/Data/Crypto/Bybit/infos.json', 'w') as f:
        json.dump(dic,f)
        f.close()
        
init_trades()

In [45]:
dowload_monthly_trading(symbol='BTCUSD', year=2020, month=5, print_mode=False)

In [46]:
df = pd.read_csv('D://Trading/Data/Crypto/Bybit/trades/BTCUSD/1min/2020/2020-05.csv.gz')
df.loc[:,'date'] = pd.to_datetime(df.time, unit='s')
df

,time,side,size,price,tickDirection,foreignNotional,date
0,1.588378e+09,Buy,111,8831.5,ZeroMinusTick,0.012569,2020-05-01 23:59:58.390206976
1,1.588378e+09,Buy,1410,8831.5,PlusTick,0.159656,2020-05-01 23:59:58.305577984
2,1.588378e+09,Sell,475,8831.0,MinusTick,0.053788,2020-05-01 23:59:58.032474112
3,1.588378e+09,Buy,111,8831.5,PlusTick,0.012569,2020-05-01 23:59:57.340058112
4,1.588378e+09,Sell,2500,8831.0,ZeroMinusTick,0.283094,2020-05-01 23:59:54.425626112
...,...,...,...,...,...,...,...
6439357,1.590883e+09,Buy,1111,9702.5,ZeroMinusTick,0.114507,2020-05-31 00:00:01.964585984
6439358,1.590883e+09,Buy,2286,9702.5,ZeroMinusTick,0.235609,2020-05-31 00:00:01.722912000
6439359,1.590883e+09,Buy,2205,9702.5,ZeroMinusTick,0.227261,2020-05-31 00:00:01.717868032
6439360,1.590883e+09,Buy,50,9702.5,ZeroMinusTick,0.005153,2020-05-31 00:00:01.702946048


## loop

In [60]:
dataType = 'trading'
for symbol in get_list_symbols(dataType):
    dic_year_month = {}
    list_filename = get_list_filename_trading(symbol)
    print(f'{datetime.datetime.now()} - dowloading {symbol} data...')
    for filename in list_filename:
        date = filename_to_date(filename, symbol)
        if date.year not in dic_year_month:
            dic_year_month[date.year] = []
        if date.month not in dic_year_month[date.year]:
            dic_year_month[date.year].append(date.month)
    del dic_year_month[date.year][-1]
            
    for year in dic_year_month:
        for month in dic_year_month[year]:
            dowload_monthly_trading(symbol=symbol, year=year, month=month, print_mode=True)     

2021-03-22 19:44:38.682152 - dowloading ADAUSDT data...
2021-03-22 19:44:39.142469 - dowloading BCHUSDT data...
2021-03-22 19:44:39.143467 downloading data for trades index BCHUSDT year 2020 and month 12
BCHUSDT2020-12-01.csv.gz not in historical data
2021-03-22 19:44:39.298286 downloading data for trades index BCHUSDT year 2021 and month 1
2021-03-22 19:45:34.848219 downloading data for trades index BCHUSDT year 2021 and month 2
2021-03-22 19:46:27.663891 - dowloading BTCUSD data...
2021-03-22 19:46:27.668904 downloading data for trades index BTCUSD year 2019 and month 10
2021-03-22 19:48:08.532092 downloading data for trades index BTCUSD year 2019 and month 11
2021-03-22 19:49:46.252467 downloading data for trades index BTCUSD year 2019 and month 12
2021-03-22 19:51:42.712456 downloading data for trades index BTCUSD year 2020 and month 1
2021-03-22 19:54:04.954922 downloading data for trades index BTCUSD year 2020 and month 2
2021-03-22 19:56:21.552958 downloading data for trades ind

In [98]:
for symbol in os.listdir('D://Trading/Data/Crypto/Bybit/trades'):
    import shutil

    shutil.rmtree(f'D://Trading/Data/Crypto/Bybit/trades/{symbol}/1min')

# custom 1min ohlc

## infos gathering

In [114]:
def update_json(assetType, broker, instrument, timeframe, symbol, columns, dic_resamp, symbol_infos):
    if f'{broker}.json' in os.listdir(f'D://Trading/Data/{assetType}'):
        with open(f'D://Trading/Data/{assetType}/{broker}.json','r') as f:
            dic = json.load(f)
            f.close()
    else:
        dic = {}
    if instrument not in dic:
        dic[instrument] = {}
    if timeframe not in dic[instrument]:
        dic[instrument][timeframe] = {}
    dic[instrument][timeframe][symbol] = {
        "columns": columns,
        "resampling_dict": dic_resamp,
        "infos": symbol_infos
    }
    with open(f'D://Trading/Data/{assetType}/{broker}.json','w') as f:
        json.dump(dic, f)
        f.close()

def update_infos(assetType, broker, instrument, timeframe, symbol, df, markets_bybit):
    columns = [col for col in df.columns]    
    list_resamp_first_col = ['time', 'open', 'ask_open', 'bid_open']
    list_resamp_last_col = ['close', 'ask_close', 'bid_close']
    list_resamp_max_col = ['high', 'ask_high', 'bid_high']
    list_resamp_min_col = ['low', 'ask_low', 'bid_low']
    list_resamp_sum_col = ['size', 'foreignNotional', 'v_b', 'v_s', 'v_weigted_p', 'v_weigted_p_b', 'v_weigted_p_s', 'qty', 'qty_b', 'qty_s']
    list_resamp_mean_col = ['v_b_over_v', 'v_s_over_v', 'p_mean', 'v_weigted_p_over_p_mean', 'v_weigted_p_b_over_v_weigted_p', 'v_weigted_p_s_over_v_weigted_p', 'p_std', 'p_std_b', 'p_std_s', 'spread']
    dic_resamp = {}
    for col in df.columns:
        if col in list_resamp_first_col:
            dic_resamp[col] = 'first'
        elif col in list_resamp_last_col:
            dic_resamp[col] = 'last'
        elif col in list_resamp_max_col:
            dic_resamp[col] = 'max'
        elif col in list_resamp_min_col:
            dic_resamp[col] = 'min'
        elif col in list_resamp_sum_col:
            dic_resamp[col] = 'sum'
        elif col in list_resamp_mean_col:
            dic_resamp[col] = 'mean'
        else:
            dic_resamp[col] = 'mean'
            
    bybit_info = markets_bybit[symbol]
    symbol_infos = {'point':bybit_info['precision']['price'], 
                    'contract_value_in_usd':bybit_info['precision']['amount'], 
                    'volume_step':bybit_info['limits']['amount']['min'], 
                    'volume_max':bybit_info['limits']['amount']['max'], 
                    'min_point_stop':0,
                    'taker_fee':bybit_info['taker'],
                    'maker_fee':bybit_info['maker'],
                   }
    
    
    update_json(assetType, broker, instrument, timeframe, symbol, columns, dic_resamp, symbol_infos)

## unit functions

In [115]:
import json, os, gzip
import datetime
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def custom_transfo(x):
    v = x['size'].ffill()
    p = x['price'].ffill()
    d = x['side'].ffill()
    
    qty = len(d) # nbs trades
    
    if qty > 0:
    
        v_b = v[d=='Buy'].sum() # volume of buy order
        v_s = v[d=='Sell'].sum() # volume of sell order

        if v_b + v_s > 0:
            v_b_over_v = v_b / (v_b + v_s)
            v_s_over_v = 1 - v_b_over_v
        else:
            v_b_over_v = 0
            v_s_over_v = 0

        p_mean = p.mean() # average price

        v_weigted_p = (v*p).sum() / v.sum() # volume averaged price
        if v_b > 0:
            v_weigted_p_b = (v*p)[d=='Buy'].sum() / v_b # volume averaged price buy side
        else:
            v_weigted_p_b = 10
        if v_s > 0:
            v_weigted_p_s = (v*p)[d=='Sell'].sum() / v_s # volume averaged price sell side
        else:
            v_weigted_p_s = 10

        v_weigted_p_over_p_mean = v_weigted_p / p_mean
        v_weigted_p_b_over_v_weigted_p = v_weigted_p_b / v_weigted_p
        v_weigted_p_s_over_v_weigted_p = v_weigted_p_s / v_weigted_p

        p_std = p.std() # std price
        p_std_b = p[d=='Buy'].std() # std buy side prices
        p_std_s = p[d=='Sell'].std() # std sell side prices

        qty = len(d) # nbs trades
        qty_b = np.array([d=='Buy']).sum()
        qty_s = qty - qty_b # nbs sell trades

        list_to_append = [v_b, v_s, v_b_over_v, v_s_over_v, p_mean, v_weigted_p, v_weigted_p_b, v_weigted_p_s, v_weigted_p_over_p_mean, v_weigted_p_b_over_v_weigted_p, v_weigted_p_s_over_v_weigted_p, p_std, p_std_b, p_std_s, qty, qty_b, qty_s]     
        list_index = ['v_b', 'v_s', 'v_b_over_v', 'v_s_over_v', 'p_mean', 'v_weigted_p', 'v_weigted_p_b', 'v_weigted_p_s', 'v_weigted_p_over_p_mean', 'v_weigted_p_b_over_v_weigted_p', 'v_weigted_p_s_over_v_weigted_p', 'p_std', 'p_std_b', 'p_std_s', 'qty', 'qty_b', 'qty_s']

        return pd.Series(list_to_append, index=list_index)
    
    else:
        

        list_to_append = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]     
        list_index = ['v_b', 'v_s', 'v_b_over_v', 'v_s_over_v', 'p_mean', 'v_weigted_p', 'v_weigted_p_b', 'v_weigted_p_s', 'v_weigted_p_over_p_mean', 'v_weigted_p_b_over_v_weigted_p', 'v_weigted_p_s_over_v_weigted_p', 'p_std', 'p_std_b', 'p_std_s', 'qty', 'qty_b', 'qty_s']

        return pd.Series(list_to_append, index=list_index)



def resamp_df(symbol, year_str, month_str, markets_bybit):
    assetType = 'Crypto'
    broker = 'Bybit'
    
    if symbol in os.listdir(f'D://Trading/Data/{assetType}/{broker}/trades'):
        if year_str in os.listdir(f'D://Trading/Data/{assetType}/{broker}/trades/{symbol}/tick'):
            if f'{year_str}-{month_str}.csv.gz' in os.listdir(f'D://Trading/Data/{assetType}/{broker}/trades/{symbol}/tick/{year_str}'):
                

                df = pd.read_csv(f'D://Trading/Data/{assetType}/{broker}/trades/{symbol}/tick/{year_str}/{year_str}-{month_str}.csv.gz')
                df.loc[:,'date'] = pd.to_datetime(df.time, unit='s')

                resamp = df.set_index('date').resample('1min')
                groupby = df.set_index('date').groupby(pd.Grouper(freq='1min'))

                df2 = resamp.agg({'time':'first', 
                                  'price':'ohlc',
                                  'size':'sum',
                                  'foreignNotional':'sum'})
                df3 = groupby.apply(custom_transfo)

                df4 = pd.concat([df2, df3], axis=1)
                df4 = df4.rename(columns={
                    ('time', 'time'):'time',
                    ('price', 'open'): 'open',
                    ('price', 'high'): 'high',
                    ('price', 'low'): 'low',
                    ('price', 'close'): 'close',
                    ('size', 'size'): 'size',
                    ('foreignNotional', 'foreignNotional'): 'foreignNotional',
                }).reset_index(drop=True)
                
                path = f'D://Trading/Data/{assetType}/{broker}/trades/{symbol}/1min/{year_str}/{year_str}-{month_str}.csv.gz'
                
                folders_to_create = path[4:].split('/')[:-1]
                tmp_path = 'D://'
                for folder in folders_to_create:
                    if folder not in os.listdir(tmp_path):
                        os.mkdir(tmp_path+folder)
                    tmp_path += folder + '/'
                
                for col in ['open', 'high', 'low', 'close']:
                    df4.loc[:,f'ask_{col}'] = df4[f'{col}'] + markets_bybit[symbol]['precision']['price'] / 2
                    df4.loc[:,f'bid_{col}'] = df4[f'{col}'] - markets_bybit[symbol]['precision']['price'] / 2

                df4.to_csv(path, index=False)
                
                update_infos(assetType, broker, 'trades', '1min', symbol, df4, markets_bybit)
                
import ccxt
markets_bybit = {market['id']:market for market in ccxt.bybit().fetch_markets()}

In [108]:
resamp_df('XRPUSD', '2020', '05', markets_bybit)

In [110]:
pd.read_csv('D://Trading/Data/Crypto/Bybit/trades/XRPUSD/1min/2020/2020-05.csv.gz')

,time,open,high,low,close,size,foreignNotional,v_b,v_s,v_b_over_v,...,qty_b,qty_s,ask_open,bid_open,ask_high,bid_high,ask_low,bid_low,ask_close,bid_close
0,1.588291e+09,0.2116,0.2118,0.2114,0.2118,13738,64964.465801,4689.0,9049.0,0.341316,...,7.0,2.0,0.21165,0.21155,0.21185,0.21175,0.21145,0.21135,0.21185,0.21175
1,1.588291e+09,0.2118,0.2125,0.2118,0.2125,59250,279114.164034,59076.0,174.0,0.997063,...,31.0,3.0,0.21185,0.21175,0.21255,0.21245,0.21185,0.21175,0.21255,0.21245
2,1.588291e+09,0.2123,0.2124,0.2120,0.2120,6879,32415.401708,6720.0,159.0,0.976886,...,3.0,5.0,0.21235,0.21225,0.21245,0.21235,0.21205,0.21195,0.21205,0.21195
3,1.588291e+09,0.2120,0.2124,0.2120,0.2123,46868,220863.179564,40181.0,6687.0,0.857323,...,3.0,5.0,0.21205,0.21195,0.21245,0.21235,0.21205,0.21195,0.21235,0.21225
4,1.588291e+09,0.2124,0.2124,0.2123,0.2123,40912,192619.882411,39929.0,983.0,0.975973,...,6.0,4.0,0.21245,0.21235,0.21245,0.21235,0.21235,0.21225,0.21235,0.21225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44634,1.590969e+09,0.2025,0.2025,0.2025,0.2025,4814,23772.839506,800.0,4014.0,0.166182,...,5.0,3.0,0.20255,0.20245,0.20255,0.20245,0.20255,0.20245,0.20255,0.20245
44635,1.590969e+09,0.2026,0.2029,0.2026,0.2029,64990,320311.618316,64911.0,79.0,0.998784,...,20.0,5.0,0.20265,0.20255,0.20295,0.20285,0.20265,0.20255,0.20295,0.20285
44636,1.590969e+09,0.2029,0.2029,0.2028,0.2029,67975,335024.540611,64975.0,3000.0,0.955866,...,9.0,2.0,0.20295,0.20285,0.20295,0.20285,0.20285,0.20275,0.20295,0.20285
44637,1.590969e+09,0.2028,0.2028,0.2026,0.2026,376,1854.050693,1.0,375.0,0.002660,...,1.0,4.0,0.20285,0.20275,0.20285,0.20275,0.20265,0.20255,0.20265,0.20255


## loop

In [111]:
dic_op_to_do = {}
for symbol in os.listdir('D://Trading/Data/Crypto/Bybit/trades'):
    if symbol not in dic_op_to_do:
        dic_op_to_do[symbol] = {}
    for year_str in os.listdir(f'D://Trading/Data/Crypto/Bybit/trades/{symbol}/tick'):
        if year_str not in dic_op_to_do[symbol]:
            dic_op_to_do[symbol][year_str] = []
        for filename in os.listdir(f'D://Trading/Data/Crypto/Bybit/trades/{symbol}/tick/{year_str}'):
            if ".csv" in filename and year_str in filename:
                month_str = filename.split('.csv')[0].split(f'{year_str}-')[1]
                if month_str not in dic_op_to_do[symbol][year_str]:
                    dic_op_to_do[symbol][year_str].append(month_str)
                    
print(dic_op_to_do)

{'BCHUSDT': {'2021': ['01', '02']}, 'BTCUSD': {'2019': ['10', '11', '12'], '2020': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '2021': ['01', '02']}, 'BTCUSDT': {'2020': ['04', '05', '06', '07', '08', '09', '10', '11', '12'], '2021': ['01', '02']}, 'EOSUSD': {'2019': ['10', '11', '12'], '2020': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '2021': ['01', '02']}, 'ETHUSD': {'2019': ['10', '11', '12'], '2020': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '2021': ['01', '02']}, 'ETHUSDT': {'2020': ['11', '12'], '2021': ['01', '02']}, 'LINKUSDT': {'2020': ['11', '12'], '2021': ['01', '02']}, 'LTCUSDT': {'2020': ['11', '12'], '2021': ['01', '02']}, 'XRPUSD': {'2019': ['10', '11', '12'], '2020': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], '2021': ['01', '02']}, 'XTZUSDT': {'2020': ['11', '12'], '2021': ['01', '02']}}


In [116]:
for symbol in dic_op_to_do.keys():
    for year_str in dic_op_to_do[symbol].keys():
        for month_str in dic_op_to_do[symbol][year_str]:
            print(f'{datetime.datetime.now()} - resampling {symbol} of month {year_str}-{month_str}')
            try:
                resamp_df(symbol, year_str, month_str, markets_bybit)
            except:
                print('error',symbol, year_str, month_str)
            

2021-03-28 10:01:08.691012 - resampling BCHUSDT of month 2021-01
2021-03-28 10:03:09.620791 - resampling BTCUSD of month 2019-10
2021-03-28 10:05:12.122352 - resampling BTCUSDT of month 2020-04
2021-03-28 10:07:00.438839 - resampling EOSUSD of month 2019-10
2021-03-28 10:08:47.121651 - resampling ETHUSD of month 2019-10
2021-03-28 10:10:38.099102 - resampling ETHUSDT of month 2020-11
2021-03-28 10:12:30.596020 - resampling LINKUSDT of month 2020-11
2021-03-28 10:14:28.854859 - resampling LTCUSDT of month 2020-11
2021-03-28 10:16:21.919617 - resampling XRPUSD of month 2019-10
2021-03-28 10:18:10.196353 - resampling XTZUSDT of month 2020-11
